In [1]:
import pandas as pd ## Обновить pandas до последней версии
import numpy as np
import math
import os
import glob
import mysql.connector
import datetime
from mysql.connector import errorcode
from sklearn import preprocessing
from sklearn import metrics

import matplotlib.pyplot as plt
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
from datetime import date
import string
from IPython.display import display, HTML

from os import listdir
from os.path import isfile, join


import warnings
warnings.filterwarnings("ignore")

In [2]:
my_path = input("Введите путь папки, в которой будут лежать все ваши файлы (пример формата C:\\Users\\User\\Documents\\Python Scripts\\IT\\Untitled Folder): ")
while my_path  == '':
    print("Ошибка: вы не ввели путь папки")
    my_path = input("Введите путь папки, в которой будут лежать все ваши файлы (пример формата C:\\Users\\User\\Documents\\Python Scripts\\IT\\Untitled Folder): ")
print("Вы ввели '" + str(my_path)+"'")

print("Введите данные для использования MySQL")
user = input("Введите имя пользователя (user name): ")
while user  == '':
    print("Ошибка: вы не ввели имя пользователя")
    user = input("Введите имя пользователя (user name): ")
print("Вы ввели '" + str(user)+"'")
database = input("Введите название базы данных (database name): ")
while database  == '':
    print("Ошибка: вы не ввели название базы данных")
    database = input("Введите название базы данных (database name): ")
print("Вы ввели '" + str(database)+"'")

Введите путь папки, в которой будут лежать все ваши файлы (пример формата C:\Users\User\Documents\Python Scripts\IT\Untitled Folder): C:\Users\plaps\Desktop\IT\project\Untitled Folder
Вы ввели 'C:\Users\plaps\Desktop\IT\project\Untitled Folder'
Введите данные для использования MySQL
Введите имя пользователя (user name): project_user
Вы ввели 'project_user'
Введите название базы данных (database name): project_db
Вы ввели 'project_db'


#### Блок Калинина Артёма

In [3]:
cnx = mysql.connector.connect(host = 'localhost', database = database, user = user)
cursor = cnx.cursor(buffered=True)

payments_table = pd.read_sql_query("SELECT * FROM payments", cnx)
payments_table.keys()

Index(['contract_number', 'payment_date', 'amount_due', 'amount_paid'], dtype='object')

In [4]:
def get_90_day_default(contract_number):
    table = payments_table[payments_table['contract_number'] == contract_number]## Выбираем строки с конкретным номером контракта
    table = table.reset_index() #обнуляем индексы, чтобы при дальнейшей итерации всё работало с 0
    for i in  range(0, len(table['amount_paid'])):
        try:
            if (table['amount_paid'][i] < table['amount_due'][i] and table['amount_paid'][i+1] < table['amount_due'][i+1] and
                table['amount_paid'][i+2] < table['amount_due'][i+2] and table['amount_paid'][i+3] < table['amount_due'][i+3]):
                #Проверяем чтобы заёмщик не платил 4 даты подряд
                date_90 = table['payment_date'][i + 3] #фиксируем дату просрочки
                date_90 = (str(pd.to_datetime(date_90).year) + '-' + str(pd.to_datetime(date_90).month) + 
                           '-' + str(pd.to_datetime(date_90).day)) #переводим в нужный формат
                break
                
            else:
                date_90 = '0000-00-00' #если нет просрочки то передаём такую дату
        except:
            True
    return date_90

In [5]:
data = []
for i in payments_table['contract_number'].unique():
    if get_90_day_default(i) == '0000-00-00':
        data.append((i, get_90_day_default(i), 0))
    else:
        data.append((i, get_90_day_default(i), 1))
data

[(100524, '0000-00-00', 0),
 (100552, '0000-00-00', 0),
 (100493, '0000-00-00', 0),
 (100797, '0000-00-00', 0),
 (100929, '0000-00-00', 0),
 (100022, '0000-00-00', 0),
 (100127, '0000-00-00', 0),
 (100520, '0000-00-00', 0),
 (100190, '0000-00-00', 0),
 (100600, '0000-00-00', 0),
 (100932, '0000-00-00', 0),
 (100582, '0000-00-00', 0),
 (100990, '0000-00-00', 0),
 (100869, '0000-00-00', 0),
 (100614, '0000-00-00', 0),
 (100323, '0000-00-00', 0),
 (100997, '0000-00-00', 0),
 (100765, '0000-00-00', 0),
 (100122, '0000-00-00', 0),
 (100068, '0000-00-00', 0),
 (100414, '0000-00-00', 0),
 (100610, '0000-00-00', 0),
 (100703, '0000-00-00', 0),
 (100684, '2017-12-14', 1),
 (100620, '0000-00-00', 0),
 (100506, '0000-00-00', 0),
 (100176, '0000-00-00', 0),
 (100431, '0000-00-00', 0),
 (100206, '0000-00-00', 0),
 (100851, '0000-00-00', 0),
 (100743, '0000-00-00', 0),
 (100518, '0000-00-00', 0),
 (100211, '0000-00-00', 0),
 (100476, '0000-00-00', 0),
 (100321, '0000-00-00', 0),
 (100297, '0000-00-0

In [13]:
cnx = mysql.connector.connect(host = 'localhost', database = database, user = user)
cursor = cnx.cursor(buffered=True)

cursor.execute("""ALTER TABLE contracts ADD default_ INT """)

for i in data:
    cursor.execute("""UPDATE contracts SET default_ = %d WHERE contract_number = %d """ %(i[2], i[0]))

#'0000-00-00' Означает что нет просрочки

cnx.commit()

ProgrammingError: 1060 (42S21): Duplicate column name 'default_'

In [14]:
applications_defaults_and_id = pd.read_sql_query("""SELECT contracts.id, age, car_owner, children, education, employed_by, family, 
gender, house_ownership, housing, income, income_type, marital_status, position, default_
FROM applications
INNER JOIN contracts ON
contracts.id = applications.id;""", cnx)
applications_defaults_and_id.head()

id  age  age_of_car  children  education  employed_by  family  gender  \
0  100009   40        17.0         1          0            1       3       0   
1  100047   50         NaN         0          2            1       2       1   
2  100025   40        14.0         1          2           20       3       0   
3  100064   60         NaN         0          2           25       1       0   
4  100023   33         NaN         1          0            9       2       0   

   house_ownership  housing  income  income_type  marital_status  position  \
0                1        0  171000            0               1         1   
1                1        0  202500            0               1         6   
2                1        0  202500            0               1        10   
3                0        1   67500            1               3         0   
4                1        0   90000            2               3         4   

   default_  
0         0  
1         1  
2         0  
3         0  
4         0

In [15]:
defaults = applications_defaults_and_id.default_
defaults

0     0
1     1
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    1
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
39    1
40    1
41    0
42    0
43    0
44    0
45    0
46    0
47    0
48    0
49    0
50    0
51    0
52    1
53    0
54    0
55    0
56    0
57    0
58    0
59    0
Name: default_, dtype: int64

id  default_
0   100009         0
1   100047         1
2   100025         0
3   100064         0
4   100023         0
5   100010         0
6   100034         0
7   100012         0
8   100036         0
9   100041         0
10  100046         0
11  100048         0
12  100052         0
13  100045         0
14  100020         0
15  100059         0
16  100049         1
17  100062         0
18  100026         0
19  100035         0
20  100073         0
21  100069         0
22  100075         0
23  100044         0
24  100006         0
25  100033         0
26  100054         0
27  100040         0
28  100011         0
29  100003         0
30  100071         0
31  100004         0
32  100016         0
33  100063         0
34  100014         0
35  100027         0
36  100019         0
37  100032         0
38  100051         0
39  100056         1
40  100031         1
41  100055         0
42  100030         0
43  100061         0
44  100039         0
45  100022         0
46  100007         0
47  100072         0
48  100037         0
49  100070         0
50  100018         0
51  100058         0
52  100050         1
53  100008         0
54  100015         0
55  100060         0
56  100053         0
57  100017         0
58  100021         0
59  100068         0

In [16]:
applications_defaults = applications_defaults_and_id.drop(['default_', 'id'], axis = 1)

### Правила обработки информации:
 1. Каждой категории текстовых полей будет присваиваться балл
 2. Непрерывные переменные сначала группируются по 25% выборки

In [17]:
#defaults = applications_defaults.default_
#defaults

AttributeError: 'DataFrame' object has no attribute 'default_'

In [19]:
max_bin = 20
force_bin = 4

# define a binning function
def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def woe_iv_categorical(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            conv = char_bin(target, df1[i])
            conv["VAR_NAME"] = i
            count = count + 1
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [20]:
applications_defaults_1  = applications_defaults.copy()

In [21]:
applications_defaults_1['age'].describe()

count    60.000000
mean     45.316667
std      12.155060
min      21.000000
25%      34.750000
50%      46.500000
75%      54.250000
max      71.000000
Name: age, dtype: float64

In [22]:
applications_defaults_1['car_owner'].describe()

count    16.000000
mean     10.312500
std       7.551766
min       1.000000
25%       3.000000
50%       9.000000
75%      14.750000
max      26.000000
Name: age_of_car, dtype: float64

In [23]:
applications_defaults_1['income'].describe()

count        60.000000
mean     171677.816667
std      113656.314964
min       38419.000000
25%       96750.000000
50%      126000.000000
75%      202500.000000
max      540000.000000
Name: income, dtype: float64

In [24]:
applications_defaults_1['age'] = np.where( (applications_defaults_1['age'] <= 34) & (applications_defaults_1['age'] >= 21), 0, 
                               np.where ((applications_defaults_1['age'] <= 46) & (applications_defaults_1['age'] > 34), 1,
                                        np.where ((applications_defaults_1['age'] <= 54) & (applications_defaults_1['age'] > 46), 2, 3)))

In [25]:
applications_defaults_1['income'] = np.where( (applications_defaults_1['income'] <= 96750) & (applications_defaults_1['income'] >= 38419), 0, 
                               np.where ((applications_defaults_1['income'] <= 126000) & (applications_defaults_1['income'] > 96750), 1,
                                        np.where ((applications_defaults_1['income'] <= 202500) & (applications_defaults_1['income'] > 126000), 2, 3)))

In [27]:
applications_defaults_1.head(5)

age  age_of_car  children  education  employed_by  family  gender  \
0    1           3         1          0            1       3       0   
1    2           3         0          2            1       2       1   
2    1           2         1          2           20       3       0   
3    3           3         0          2           25       1       0   
4    0           3         1          0            9       2       0   

   house_ownership  housing  income  income_type  marital_status  position  
0                1        0       2            0               1         1  
1                1        0       2            0               1         6  
2                1        0       2            0               1        10  
3                0        1       0            1               3         0  
4                1        0       0            2               3         4

In [28]:
woe_iv_cat_df = woe_iv_categorical(applications_defaults_1, defaults)[0]
print('Полученная таблица WOE и IV по всем переменным:')
display(woe_iv_cat_df)

Полученная таблица WOE и IV по всем переменным:


VAR_NAME  MIN_VALUE  MAX_VALUE  COUNT  EVENT  EVENT_RATE  NONEVENT  \
0               age          0          0     15      0    0.000000        15   
1               age          1          1     15      1    0.066667        14   
2               age          2          2     15      3    0.200000        12   
3               age          3          3     15      1    0.066667        14   
4        age_of_car          0          0      5      1    0.200000         4   
5        age_of_car          1          1      3      0    0.000000         3   
6        age_of_car          2          2      4      0    0.000000         4   
7        age_of_car          3          3     48      4    0.083333        44   
8          children          0          0     47      5    0.106383        42   
9          children          1          1     12      0    0.000000        12   
10         children          2          2      1      0    0.000000         1   
11        education          0          0     18      1    0.055556        17   
12        education          1          1      1      0    0.000000         1   
13        education          2          2     41      4    0.097561        37   
14      employed_by          0          0      3      0    0.000000         3   
15      employed_by          1          1     11      2    0.181818         9   
16      employed_by          2          2      1      0    0.000000         1   
17      employed_by          3          3      1      0    0.000000         1   
18      employed_by          4          4      1      0    0.000000         1   
19      employed_by          5          5      3      0    0.000000         3   
20      employed_by          6          6      1      0    0.000000         1   
21      employed_by          7          7      1      0    0.000000         1   
22      employed_by          8          8      2      0    0.000000         2   
23      employed_by          9          9      1      0    0.000000         1   
24      employed_by         10         10      2      0    0.000000         2   
25      employed_by         11         11      1      0    0.000000         1   
26      employed_by         12         12      2      0    0.000000         2   
27      employed_by         13         13      1      0    0.000000         1   
28      employed_by         14         14      2      0    0.000000         2   
29      employed_by         15         15      1      0    0.000000         1   
..              ...        ...        ...    ...    ...         ...       ...   
46  house_ownership          1          1     46      4    0.086957        42   
47          housing          0          0     55      5    0.090909        50   
48          housing          1          1      1      0    0.000000         1   
49          housing          2          2      2      0    0.000000         2   
50          housing          3          3      2      0    0.000000         2   
51           income          0          0     15      0    0.000000        15   
52           income          1          1     15      2    0.133333        13   
53           income          2          2     17      2    0.117647        15   
54           income          3          3     13      1    0.076923        12   
55      income_type          0          0      8      1    0.125000         7   
56      income_type          1          1      9      1    0.111111         8   
57      income_type          2          2      8      0    0.000000         8   
58      income_type          3          3     35      3    0.085714        32   
59   marital_status          0          0      8      1    0.125000         7   
60   marital_status          1          1     36      2    0.055556        34   
61   marital_status          2          2      1      0    0.000000         1   
62   marital_status          3          3     12      1    0.083333        11   
63   marital_status          4          4 

### Правило присвоения баллов: 
 1. Ранжируем категории в каждой переменной по возрастанию WOE
 2. Самому низкому WOE назначается балл равный одному - далее по возрастанию WOE каждой категории добавляется три балла
 3. Полученные таким образом баллы нормализуются к шкале от 1 до 100: количество баллов конкретной категории делится на сумму уникальных баллов и умножается на 100%

In [29]:
def get_scores(var_name):
    count = 0
    values = [0]
    for i in range(0, woe_iv_cat_df[woe_iv_cat_df['VAR_NAME'] == var_name].sort_values(['WOE']).shape[0] - 1):
        table = woe_iv_cat_df[woe_iv_cat_df['VAR_NAME'] == var_name].sort_values(['WOE'])
        if table.iloc[i][10] == table.iloc[i + 1][10]:
            values.append(count)
        else:
            count = count + 3
            values.append(count)
    c = table
    c['scores'] = values
    return c

In [30]:
new_table = pd.DataFrame()
for i in woe_iv_cat_df['VAR_NAME'].unique():
    new_table = new_table.append(get_scores(i))

In [31]:
our_dict = []
for i in new_table['VAR_NAME'].unique():
    old_keys = new_table[new_table['VAR_NAME'] == i].MIN_VALUE
    new_scores = new_table[new_table['VAR_NAME'] == i].scores
    dictionary = dict(zip(old_keys, new_scores))
    our_dict.append((i, dictionary))

In [32]:
our_dict

[('age', {1: 0, 3: 0, 0: 3, 2: 6}),
 ('age_of_car', {1: 0, 2: 0, 3: 0, 0: 3}),
 ('children', {1: 0, 2: 0, 0: 3}),
 ('education', {0: 0, 1: 3, 2: 6}),
 ('employed_by',
  {0: 0,
   23: 0,
   20: 0,
   19: 0,
   18: 0,
   16: 0,
   15: 0,
   14: 0,
   13: 0,
   24: 0,
   11: 0,
   12: 0,
   9: 0,
   8: 0,
   7: 0,
   6: 0,
   5: 0,
   4: 0,
   3: 0,
   2: 0,
   10: 0,
   25: 3,
   17: 6,
   1: 9,
   21: 12}),
 ('family', {3: 0, 4: 0, 2: 3, 1: 6}),
 ('gender', {1: 0, 0: 3}),
 ('house_ownership', {0: 0, 1: 3}),
 ('housing', {1: 0, 2: 0, 3: 0, 0: 3}),
 ('income', {3: 0, 0: 3, 2: 6, 1: 9}),
 ('income_type', {2: 0, 3: 3, 1: 6, 0: 9}),
 ('marital_status', {1: 0, 2: 3, 3: 3, 0: 6, 4: 9}),
 ('position',
  {0: 0, 1: 3, 2: 3, 3: 3, 4: 3, 5: 3, 7: 3, 8: 3, 9: 3, 11: 3, 6: 6, 10: 9})]

In [33]:
for i in our_dict:
    applications_defaults_1[i[0]] = applications_defaults_1[i[0]].map(i[1] )
applications_defaults_1.head()

age  age_of_car  children  education  employed_by  family  gender  \
0    0           0         0          0            9       0       3   
1    6           0         3          6            9       3       0   
2    0           0         0          6            0       0       3   
3    0           0         3          6            3       6       3   
4    3           0         0          0            0       3       3   

   house_ownership  housing  income  income_type  marital_status  position  
0                3        3       6            9               0         3  
1                3        3       6            9               0         6  
2                3        3       6            9               0         9  
3                0        0       3            6               3         0  
4                3        3       3            0               3         3

In [34]:
for i in applications_defaults_1:
    applications_defaults_1[i] = [(x / applications_defaults_1[i].unique().sum() * 100) for x in applications_defaults_1[i]]

In [35]:
applications_defaults_1

age  age_of_car  children  education  employed_by     family  \
0    0.000000         0.0       0.0   0.000000         30.0   0.000000   
1   66.666667         0.0     100.0  66.666667         30.0  33.333333   
2    0.000000         0.0       0.0  66.666667          0.0   0.000000   
3    0.000000         0.0     100.0  66.666667         10.0  66.666667   
4   33.333333         0.0       0.0   0.000000          0.0  33.333333   
5   66.666667         0.0     100.0   0.000000          0.0  33.333333   
6   33.333333         0.0     100.0   0.000000         30.0  66.666667   
7    0.000000         0.0     100.0  66.666667          0.0  66.666667   
8   33.333333         0.0     100.0  66.666667          0.0  33.333333   
9   33.333333         0.0     100.0   0.000000         20.0  33.333333   
10   0.000000       100.0     100.0   0.000000         30.0  33.333333   
11  66.666667         0.0     100.0  66.666667          0.0  33.333333   
12  33.333333         0.0       0.0  66.666667         20.0   0.000000   
13   0.000000         0.0     100.0  66.666667         10.0  33.333333   
14   0.000000         0.0     100.0  66.666667          0.0  33.333333   
15  66.666667         0.0       0.0   0.000000          0.0   0.000000   
16   0.000000         0.0     100.0  66.666667         20.0  33.333333   
17  66.666667         0.0     100.0  66.666667         10.0  33.333333   
18  33.333333         0.0       0.0  66.666667         20.0   0.000000   
19   0.000000         0.0     100.0  66.666667         30.0  33.333333   
20   0.000000         0.0     100.0   0.000000         10.0  33.333333   
21   0.000000         0.0       0.0  66.666667          0.0  33.333333   
22  33.333333         0.0     100.0   0.000000          0.0  66.666667   
23   0.000000         0.0     100.0  66.666667          0.0  33.333333   
24  66.666667         0.0     100.0  66.666667         30.0  33.333333   
25  33.333333       100.0     100.0   0.000000          0.0  66.666667   
26   0.000000         0.0     100.0  66.666667         30.0  33.333333   
27  66.666667         0.0     100.0   0.000000          0.0  33.333333   
28   0.000000         0.0     100.0  66.666667         10.0  33.333333   
29  66.666667         0.0     100.0   0.000000          0.0  33.333333   
30   0.000000         0.0     100.0  66.666667          0.0  33.333333   
31  66.666667         0.0     100.0  66.666667          0.0  66.666667   
32   0.000000         0.0     100.0  66.666667          0.0  33.333333   
33   0.000000         0.0     100.0   0.000000          0.0  33.333333   
34  33.333333         0.0       0.0   0.000000          0.0   0.000000   
35   0.000000         0.0     100.0  66.666667         10.0  33.333333   
36  33.333333         0.0     100.0  66.666667          0.0  66.666667   
37  66.666667         0.0       0.0  66.666667          0.0   0.000000   
38  33.333333         0.0     100.0  66.666667         30.0  33.333333   
39  66.666667       100.0     100.0  66.666667         40.0  33.333333   
40  66.666667         0.0     100.0  66.666667         30.0  66.666667   
41   0.000000         0.0     100.0  66.666667          0.0  33.333333   
42   0.000000         0.0     100.0  66.666667         30.0  33.333333   
43  33.333333         0.0       0.0   0.000000         20.0   0.000000   
44  33.333333       100.0       0.0  66.666667         20.0   0.000000   
45  66.666667         0.0     100.0  66.666667          0.0  66.666667   
46   0.000000         0.0     100.0  66.666667          0.0  66.666667   
47  33.333333         0.0     100.0  33.333333          0.0  66.666667   
48   0.000000         0.0     100.0  66.666667          0.0  33.333333   
49   0.000000       100.0     100.0   0.000000         30.0  66.666667   
50   0.000000         0.0     100.0  66.666667         40.0  33.333333   
51   0.000000         0.0     100.0   0.000000          0.0  33.333333   
52   0.000000         0.0     100.0   0.000000         10.0  66.666667   
53  66.666667 

In [36]:
applications_defaults_1['id'] = applications_defaults_and_id['id']
cols = list(applications_defaults_1)
cols.insert(0, cols.pop(cols.index('id')))
applications_defaults_1 = applications_defaults_1.ix[:, cols]
print('Полученная таблица со скорингами по каждой переменной для каждого заемщика:')
display(applications_defaults_1)

Полученная таблица со скорингами по каждой переменной для каждого заемщика:


id        age  age_of_car  children  education  employed_by  \
0   100009   0.000000         0.0       0.0   0.000000         30.0   
1   100047  66.666667         0.0     100.0  66.666667         30.0   
2   100025   0.000000         0.0       0.0  66.666667          0.0   
3   100064   0.000000         0.0     100.0  66.666667         10.0   
4   100023  33.333333         0.0       0.0   0.000000          0.0   
5   100010  66.666667         0.0     100.0   0.000000          0.0   
6   100034  33.333333         0.0     100.0   0.000000         30.0   
7   100012   0.000000         0.0     100.0  66.666667          0.0   
8   100036  33.333333         0.0     100.0  66.666667          0.0   
9   100041  33.333333         0.0     100.0   0.000000         20.0   
10  100046   0.000000       100.0     100.0   0.000000         30.0   
11  100048  66.666667         0.0     100.0  66.666667          0.0   
12  100052  33.333333         0.0       0.0  66.666667         20.0   
13  100045   0.000000         0.0     100.0  66.666667         10.0   
14  100020   0.000000         0.0     100.0  66.666667          0.0   
15  100059  66.666667         0.0       0.0   0.000000          0.0   
16  100049   0.000000         0.0     100.0  66.666667         20.0   
17  100062  66.666667         0.0     100.0  66.666667         10.0   
18  100026  33.333333         0.0       0.0  66.666667         20.0   
19  100035   0.000000         0.0     100.0  66.666667         30.0   
20  100073   0.000000         0.0     100.0   0.000000         10.0   
21  100069   0.000000         0.0       0.0  66.666667          0.0   
22  100075  33.333333         0.0     100.0   0.000000          0.0   
23  100044   0.000000         0.0     100.0  66.666667          0.0   
24  100006  66.666667         0.0     100.0  66.666667         30.0   
25  100033  33.333333       100.0     100.0   0.000000          0.0   
26  100054   0.000000         0.0     100.0  66.666667         30.0   
27  100040  66.666667         0.0     100.0   0.000000          0.0   
28  100011   0.000000         0.0     100.0  66.666667         10.0   
29  100003  66.666667         0.0     100.0   0.000000          0.0   
30  100071   0.000000         0.0     100.0  66.666667          0.0   
31  100004  66.666667         0.0     100.0  66.666667          0.0   
32  100016   0.000000         0.0     100.0  66.666667          0.0   
33  100063   0.000000         0.0     100.0   0.000000          0.0   
34  100014  33.333333         0.0       0.0   0.000000          0.0   
35  100027   0.000000         0.0     100.0  66.666667         10.0   
36  100019  33.333333         0.0     100.0  66.666667          0.0   
37  100032  66.666667         0.0       0.0  66.666667          0.0   
38  100051  33.333333         0.0     100.0  66.666667         30.0   
39  100056  66.666667       100.0     100.0  66.666667         40.0   
40  100031  66.666667         0.0     100.0  66.666667         30.0   
41  100055   0.000000         0.0     100.0  66.666667          0.0   
42  100030   0.000000         0.0     100.0  66.666667         30.0   
43  100061  33.333333         0.0       0.0   0.000000         20.0   
44  100039  33.333333       100.0       0.0  66.666667         20.0   
45  100022  66.666667         0.0     100.0  66.666667          0.0   
46  100007   0.000000         0.0     100.0  66.666667          0.0   
47  100072  33.333333         0.0     100.0  33.333333          0.0   
48  100037   0.000000         0.0     100.0  66.666667          0.0   
49  100070   0.000000       100.0     100.0   0.000000         30.0   
50  100018   0.000000         0.0     100.0  66.666667         40.0   
51  100058   0.000000         0.0     100.0   0.000000          0.0   
52  100050   0.000000         0.0     100.0   0.000000         10.0   
53  100008  66.666667         0.0     100.0  66.666667          0.0   
54  100015   0.000000         0.0     100.0  66.666667         10.0   
55  100060   0.000000         0.

In [37]:
cnx.close()

### Выгрузка из этого блока

In [48]:
applications_defaults_1.to_csv(my_path + '\\output_3\\scores_for_vars.csv', header=True)
woe_iv_cat_df.to_csv(my_path + '\\output_3\\woe_iv_table.csv', header=True)


In [51]:
applications_defaults_and_id[['id', 'default_']].to_csv(my_path + '\\output_3\\defaults_storage.csv', header=True)

In [1]:
print("""part_3 выполнена, расчитаны WOE и IF по всем переменным, рассчитаны кредитные скоры заёмщиков по каждой переменной
посмотрите на результаты в папке output_3""")

part_3 выполнена, расчитаны WOE и IF по всем переменным, рассчитаны кредитные скоры заёмщиков по каждой переменной
посмотрите на результаты в папке output_3
